From CLUSTER data mining tool, 'dm-intervals-c2-240613-093916.csv' gives time intervals when C2 was within 1000 km of the Earth's surface.

In [1]:
import cdflib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utc_jday_conv import utc, utc_diff, jd
from kamodo_ccmc.flythrough.utils import ConvertCoord
from sgp4.api import Satrec, jday
from datetime import datetime, timezone
import re
r_e = 6371. # radius of earth

### Intervals < 1000 km

In [4]:
interval = np.genfromtxt('./dm-intervals-c2-240613-093916.csv',delimiter=',',dtype='str')
interval2 = np.genfromtxt('./dm-intervals-c2-240613-093916.csv',dtype='str')
df_int = pd.to_datetime(pd.Series(interval[:,0]))
df_int = df_int.to_frame(name='dt1') # initialising dataframe with time column
dt2 = pd.to_datetime(pd.Series(interval[:,1]))
dt2 = dt2.to_frame(name='dt2')
dt_utc = pd.DataFrame({'dt1-utc': utc(interval[:,0]), 'dt2-utc': utc(interval[:,1])})
dt_jd = pd.DataFrame({'dt1-jd': jd(interval[:,0],True), 'dt2-jd': jd(interval[:,1],True)})
diff = pd.DataFrame({'Int-min': utc_diff(interval2)[:,2]/60})
df_int = df_int.join((dt2, dt_utc, dt_jd, diff))
# df_int.to_pickle('int_df.pkl')

# print(df_int)
# df_int.sort_values(by='Int-min')

### Positional data between 2010-10-24 and 2011-12-14 (CSA)

In [17]:
pos = cdflib.CDF('../posvel/C2_POSGSE_20101024_184200_20111214_205200.cdf')

t_df = pd.to_datetime(pd.Series(cdflib.cdfepoch.encode(pos['time_tags__C2_CP_AUX_POSGSE_1M'])))

t_ar = t_df.to_numpy(dtype='str')
time = np.zeros(t_ar.shape[0])

for i in range(t_ar.shape[0]):
    t_store = re.split('[T,:.-]+',t_ar[i])
    d1, d2, d3, t1, t2, t3, zero = [int(x) for x in t_store] # turns float into integer
    time[i] = datetime(d1,d2,d3,t1,t2,t3).timestamp()

# in gse
gse_pos = pd.DataFrame({'X_GSE': pos['sc_r_xyz_gse__C2_CP_AUX_POSGSE_1M'][:,0], 'Y_GSE': pos['sc_r_xyz_gse__C2_CP_AUX_POSGSE_1M'][:,1], 'Z_GSE': pos['sc_r_xyz_gse__C2_CP_AUX_POSGSE_1M'][:,2]})

t_utc_df = pd.DataFrame({'dt': time})
gse_pos = gse_pos.join(t_utc_df)
gse_pos = gse_pos.set_index('dt')

gse_pos

,X_GSE,Y_GSE,Z_GSE
dt,,,
1.287942e+09,5241.859863,-3829.040039,3500.580078
1.287942e+09,4797.600098,-4210.069824,3666.179932
1.287942e+09,4336.089844,-4575.939941,3818.570068
1.287942e+09,3859.070068,-4925.419922,3957.280029
1.287942e+09,3368.429932,-5257.500000,4081.979980
...,...,...,...
1.323896e+09,1720.939941,-5500.569824,4974.979980
1.323896e+09,1150.890015,-5636.569824,4847.850098
1.323896e+09,576.979980,-5753.589844,4704.390137


In [ ]:
# convert GSE coordinates to TEME

# interval to evaluate
int_index = 0

# defining time range for ConvertCoord
t1 = df_int.loc[int_index,'dt1-utc']
t2 = df_int.loc[int_index,'dt2-utc']
time_int = np.arange(t1,t2+60,60)

gse_int = gse_pos.loc[t1:t2,['X_GSE', 'Y_GSE', 'Z_GSE']].to_numpy()/r_e

x_teme, y_teme, z_teme = ConvertCoord(time_int,gse_int[:,0],gse_int[:,1],gse_int[:,2],'GSE','car','teme','car',verbose=False)[:-1]

print(x_teme, y_teme, z_teme)

### Checking

In [ ]:
lim = 3500
x_teme, y_teme, z_teme = ConvertCoord(time[:lim],gse_arr[:lim,0],gse_arr[:lim,1],gse_arr[:lim,2],'GSE','car','teme','car',verbose=False)[:-1]
teme_pos = pd.DataFrame({'X_TEME': x_teme, 'Y_TEME': y_teme, 'Z_TEME': z_teme})

In [ ]:
def makecubelimits(axis, centers=None, hw=None):
    lims = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()
    if centers == None:
        centers = [0.5*sum(pair) for pair in lims] 
    if hw == None:
        widths  = [pair[1] - pair[0] for pair in lims]
        hw      = 0.5*max(widths)
        ax.set_xlim(centers[0]-hw, centers[0]+hw)
        ax.set_ylim(centers[1]-hw, centers[1]+hw)
        ax.set_zlim(centers[2]-hw, centers[2]+hw)
    else:
        try:
            hwx, hwy, hwz = hw
            ax.set_xlim(centers[0]-hwx, centers[0]+hwx)
            ax.set_ylim(centers[1]-hwy, centers[1]+hwy)
            ax.set_zlim(centers[2]-hwz, centers[2]+hwz)
        except:
            ax.set_xlim(centers[0]-hw, centers[0]+hw)
            ax.set_ylim(centers[1]-hw, centers[1]+hw)
            ax.set_zlim(centers[2]-hw, centers[2]+hw)
    return centers, hw

theta = np.linspace(0, 2*np.pi, 201)
cth, sth, zth = [f(theta) for f in (np.cos, np.sin, np.zeros_like)]
lon0 = r_e*np.vstack((cth, zth, sth))
lons = []
for phi in np.pi/180*np.arange(0, 180, 45):
    cph, sph = [f(phi) for f in (np.cos, np.sin)]
    lon = np.vstack((lon0[0]*cph - lon0[1]*sph,
                     lon0[1]*cph + lon0[0]*sph,
                     lon0[2]) )
    lons.append(lon)

lat0 = r_e*np.vstack((cth, sth, zth))
lats = []
for phi in np.pi/180*np.arange(-75, 90, 45):
    cph, sph = [f(phi) for f in (np.cos, np.sin)]
    lat = r_e*np.vstack((cth*cph, sth*cph, zth+sph))
    lats.append(lat)

# plotting

fig = plt.figure(figsize=[10, 8])

ax  = fig.add_subplot(1, 1, 1, projection='3d')


ax.plot(x_teme*r_e,y_teme*r_e,z_teme*r_e)

for x, y, z in lons:
    ax.plot(x, y, z, '-k')
for x, y, z in lats:
    ax.plot(x, y, z, '-k')

# ax.view_init(135, -90)

centers, hw = makecubelimits(ax)
# plt.savefig('',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
pos.cdf_info()